In [68]:
import sys
!{sys.executable} -m pip install 'pandas'
!{sys.executable} -m pip install 'certifi' --upgrade
!{sys.executable} -m pip install 'openpyxl'
!{sys.executable} -m pip install 'sqlalchemy'
!{sys.executable} -m pip install 'pymysql'




[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.0.9_2/libexec/bin/python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.0.9_2/libexec/bin/python -m pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 5.8 MB/s eta 0:00:000:00:01

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.0.9_2/libexec/bin/python -m pip install --upgrade pip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 16.4 MB/s eta 0:00:00m eta 0:00:010:00:01

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.0.9_2/libexec/bin/python -m pip install --upgrade pip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.7 MB/s eta 0:00:00

[notice] A new release of pip is ava

In [4]:
import requests
import certifi
import pandas as pd

## The purpose of this notebook is to showcase (And troubleshoot now) a API connection to the swagger - rest api that MGL uses.

### MGL Swagger page
https://malegislature.gov/api/swagger/index.html?url=/api/swagger/v1/swagger.json

### Requests docs:
https://docs.python-requests.org/en/latest/user/advanced/#ssl-cert-verification
https://docs.python-requests.org/en/latest/user/advanced/#ssl-cert-verification

### Troubleshooting docs:
https://stackoverflow.com/questions/54533595/curl-queries-work-but-python-request-fails
https://stackoverflow.com/questions/46604114/python-requests-ssl-error-certificate-verify-failed
https://stackoverflow.com/questions/27068163/python-requests-not-handling-missing-intermediate-certificate-only-from-one-mach/66111417#66111417

### Other docs:
https://www.geeksforgeeks.org/ssl-certificate-verification-python-requests/
https://levelup.gitconnected.com/solve-the-dreadful-certificate-issues-in-python-requests-module-2020d922c72f
https://www.reddit.com/r/learnpython/comments/q3tvxa/error_when_using_proxy_with_python_requests/
https://community.esri.com/t5/python-questions/solving-ssl-errors-in-python-requests/td-p/1124005

In [2]:
!pip list

Package            Version
------------------ ---------
certifi            2023.7.22
charset-normalizer 3.2.0
distlib            0.3.7
filelock           3.12.4
idna               3.4
pip                23.2.1
pipenv             2023.9.8
platformdirs       3.10.0
requests           2.31.0
setuptools         68.2.2
urllib3            2.0.4
virtualenv         20.24.5


In [3]:
import pandas as pd
import subprocess
import json

def fetch_data_with_curl(url):

    headers = ["accept: application/json"]
    
    try:
        result = subprocess.run(['curl', '-s', '-X', 'GET', url, '-H'] + headers, 
                                capture_output=True, check=True, text=True)

        return json.loads(result.stdout)
    except subprocess.CalledProcessError as e:
        raise Exception(f"Error fetching data from {url}: {e}") from e

url = "https://malegislature.gov/api/Chapters"
data = fetch_data_with_curl(url)

df = pd.DataFrame(data)

print(df)


     Code                                     Details
0       1     http://malegislature.gov/api/Chapters/1
1       2     http://malegislature.gov/api/Chapters/2
2       3     http://malegislature.gov/api/Chapters/3
3       4     http://malegislature.gov/api/Chapters/4
4       5     http://malegislature.gov/api/Chapters/5
..    ...                                         ...
695   23N   http://malegislature.gov/api/Chapters/23N
696  115B  http://malegislature.gov/api/Chapters/115B
697   40Y   http://malegislature.gov/api/Chapters/40Y
698  176X  http://malegislature.gov/api/Chapters/176X
699  131B  http://malegislature.gov/api/Chapters/131B

[700 rows x 2 columns]


In [4]:
df['Code']

0         1
1         2
2         3
3         4
4         5
       ... 
695     23N
696    115B
697     40Y
698    176X
699    131B
Name: Code, Length: 700, dtype: object

In [6]:
# Pull and retrieve all chapters in json
chapters_request_url = "https://malegislature.gov/api/Chapters"

chapter_response = requests.get(chapters_request_url)

if chapter_response.status_code == 200:
    chapter_json = chapter_response.json()
else: chapter_response.raise_for_status()

In [6]:
# path of cert
# certifi.where()

'/opt/homebrew/opt/python-certifi/lib/python3.11/site-packages/certifi/cacert.pem'

In [7]:
# Attempt #2 with certifi
# chapter_response = requests.get(chapters_request_url, verify=certifi.where())

# if chapter_response.status_code == 200:
#     chapter_json = chapter_response.json()
# else: chapter_response.raise_for_status()

# print(chapter_json)

In [8]:
# Attempt #3 with verify off
# chapter_response = requests.get(chapters_request_url, verify=False)

# if chapter_response.status_code == 200:
#     chapter_json = chapter_response.json()
# else: chapter_response.raise_for_status()

# print(chapter_json)

In [10]:
# Create dataframe from request
df = pd.DataFrame(chapter_json)

# make a list of chapters that are relevant and apply a filter
mgl_chapters = ["90", "263", "263A", "264", "265", "266", "267", "267A", "268", "268A", "268B", "269", "270", "271", "271A", "272", "273", "273A", "274"]
filtered_df = df[df['Code'].isin(mgl_chapters)]

In [15]:
filtered_df

,Code,Details
249,90,http://malegislature.gov/api/Chapters/90
666,263,http://malegislature.gov/api/Chapters/263
667,263A,http://malegislature.gov/api/Chapters/263A
668,264,http://malegislature.gov/api/Chapters/264
669,265,http://malegislature.gov/api/Chapters/265
670,266,http://malegislature.gov/api/Chapters/266
671,267,http://malegislature.gov/api/Chapters/267
672,267A,http://malegislature.gov/api/Chapters/267A
673,268,http://malegislature.gov/api/Chapters/268
674,268A,http://malegislature.gov/api/Chapters/268A


In [ ]:
mgl_request_url = "https://malegislature.gov/api/Chapters

In [12]:
# Test of pulling all chapter 90 sections
c90_request_url = "https://malegislature.gov/api/Chapters/90/Sections"
c90_response = requests.get(c90_request_url)

if c90_response.status_code == 200:
    c90_json = c90_response.json()
else: c90_response.raise_for_status()

df2 = pd.DataFrame(c90_json)
df2



,Code,ChapterCode,Details
0,1,90,http://malegislature.gov/api/Chapters/90/Secti...
1,1A,90,http://malegislature.gov/api/Chapters/90/Secti...
2,1B,90,http://malegislature.gov/api/Chapters/90/Secti...
3,1C,90,http://malegislature.gov/api/Chapters/90/Secti...
4,1D,90,http://malegislature.gov/api/Chapters/90/Secti...
...,...,...,...
296,62,90,http://malegislature.gov/api/Chapters/90/Secti...
297,63,90,http://malegislature.gov/api/Chapters/90/Secti...
298,221/2,90,http://malegislature.gov/api/Chapters/90/Secti...
299,241/2,90,http://malegislature.gov/api/Chapters/90/Secti...


In [20]:
def retrieve_mgl_chapters(chapters):
    """
    Retrieve sections for each chapter in the Massachusetts General Laws.

    Args:
    chapters (list): A list of chapter numbers as strings.

    Returns:
    pd.DataFrame: A DataFrame containing all sections of the specified chapters.
    """
    base_url = "https://malegislature.gov/api/Chapters/{}/Sections"
    all_chapters_data = []

    for chapter in chapters:
        try:
            response = requests.get(base_url.format(chapter))
            response.raise_for_status()  # Will raise HTTPError for 4XX/5XX responses
            chapter_json = response.json()
            chapter_df = pd.DataFrame(chapter_json)
            chapter_df['Chapter'] = chapter  # Adding chapter number for reference
            all_chapters_data.append(chapter_df)
        except requests.HTTPError as http_err:
            print(f"HTTP error occurred for chapter {chapter}: {http_err}")
        except Exception as err:
            print(f"An error occurred for chapter {chapter}: {err}")

    all_chapters_df = pd.concat(all_chapters_data, ignore_index=True)
    return all_chapters_df

# Example usage
full_mgl_df = retrieve_mgl_chapters(mgl_chapters)


HTTP error occurred for chapter 273A: 400 Client Error: No General Law Sections were found. for url: https://malegislature.gov/api/Chapters/273A/Sections


In [25]:
full_mgl_df[full_mgl_df['ChapterCode'] == '90']

,Code,ChapterCode,Details,Chapter
0,1,90,http://malegislature.gov/api/Chapters/90/Secti...,90
1,1A,90,http://malegislature.gov/api/Chapters/90/Secti...,90
2,1B,90,http://malegislature.gov/api/Chapters/90/Secti...,90
3,1C,90,http://malegislature.gov/api/Chapters/90/Secti...,90
4,1D,90,http://malegislature.gov/api/Chapters/90/Secti...,90
...,...,...,...,...
296,62,90,http://malegislature.gov/api/Chapters/90/Secti...,90
297,63,90,http://malegislature.gov/api/Chapters/90/Secti...,90
298,221/2,90,http://malegislature.gov/api/Chapters/90/Secti...,90
299,241/2,90,http://malegislature.gov/api/Chapters/90/Secti...,90


In [13]:
# Test of pulling c90s10

c90s10_request_url = "https://malegislature.gov/api/Chapters/90/Sections/10"
c90s10_response = requests.get(c90s10_request_url)

if c90s10_response.status_code == 200:
    c90s10_json = c90s10_response.json()
else: c90s10_response.raise_for_status()

df3 = pd.DataFrame(c90s10_json)

In [14]:
df3

,Code,Name,IsRepealed,Text,Chapter,Part
Code,10,Operation of motor vehicle without license; me...,False,Section 10. No person under sixteen years of a...,90,I
Details,10,Operation of motor vehicle without license; me...,False,Section 10. No person under sixteen years of a...,http://malegislature.gov/api/Chapters/90,http://malegislature.gov/api/Parts/I


In [30]:
def add_section_details(df, chapter_column, code_column):
    """
    Add details for each chapter and section in the DataFrame.

    Args:
    df (pd.DataFrame): The DataFrame containing chapters and sections.
    chapter_column (str): The column name in the DataFrame for the chapter.
    code_column (str): The column name in the DataFrame for the section code.

    Returns:
    pd.DataFrame: The updated DataFrame with added section details.
    """
    base_url = "https://malegislature.gov/api/Chapters/{}/Sections/{}"

    # Create or convert 'IsRepealed' column to boolean type
    if 'IsRepealed' not in df.columns:
        df['IsRepealed'] = False
    df['IsRepealed'] = df['IsRepealed'].astype(bool)

    for index, row in df.iterrows():
        chapter = row[chapter_column]
        section = row[code_column]
        request_url = base_url.format(chapter, section)

        try:
            response = requests.get(request_url)
            response.raise_for_status()
            section_data = response.json()

            df.at[index, 'Name'] = section_data.get('Name', '')
            df.at[index, 'IsRepealed'] = section_data.get('IsRepealed', False)
            df.at[index, 'Text'] = section_data.get('Text', '')
            df.at[index, 'Part'] = section_data.get('Part', '')

        except requests.HTTPError as http_err:
            print(f"HTTP error occurred for Chapter {chapter}, Section {section}: {http_err}")
        except Exception as err:
            print(f"An error occurred for Chapter {chapter}, Section {section}: {err}")

    return df


In [31]:
full_df = add_section_details(full_mgl_df, 'Chapter', 'Code')

HTTP error occurred for Chapter 90, Section 2C: 500 Server Error: Internal Server Error for url: https://malegislature.gov/api/Chapters/90/Sections/2C
HTTP error occurred for Chapter 90, Section 4: 500 Server Error: Internal Server Error for url: https://malegislature.gov/api/Chapters/90/Sections/4
HTTP error occurred for Chapter 90, Section 7BB: 500 Server Error: Internal Server Error for url: https://malegislature.gov/api/Chapters/90/Sections/7BB
HTTP error occurred for Chapter 90, Section 7D1/2: 404 Client Error: Not Found for url: https://malegislature.gov/api/Chapters/90/Sections/7D1/2
HTTP error occurred for Chapter 90, Section 7F: 500 Server Error: Internal Server Error for url: https://malegislature.gov/api/Chapters/90/Sections/7F
HTTP error occurred for Chapter 90, Section 7N1/2: 404 Client Error: Not Found for url: https://malegislature.gov/api/Chapters/90/Sections/7N1/2
HTTP error occurred for Chapter 90, Section 7N1/4: 404 Client Error: Not Found for url: https://malegislat

In [46]:
full_df.rename(columns={'Code': 'Section'}, inplace=True)

In [48]:
compare_result = full_df['ChapterCode'] == full_df['Chapter']

In [52]:
true_count = compare_result.sum()

In [53]:
true_count

1142

In [54]:
compare_result

0       True
1       True
2       True
3       True
4       True
        ... 
1137    True
1138    True
1139    True
1140    True
1141    True
Length: 1142, dtype: bool

In [55]:
full_df.columns

Index(['Section', 'ChapterCode', 'Details', 'Chapter', 'Name', 'IsRepealed',
       'Text', 'Part'],
      dtype='object')

In [56]:
full_df.drop(columns='Chapter', inplace=True)

In [57]:
full_df.rename(columns={'ChapterCode': 'Chapter'}, inplace=True)


In [58]:
full_df.columns

Index(['Section', 'Chapter', 'Details', 'Name', 'IsRepealed', 'Text', 'Part'], dtype='object')

In [59]:
new_order = ['Chapter', 'Section', 'Name', 'Text', 'IsRepealed', 'Details', 'Part']

full_df = full_df[new_order]

In [62]:
full_df.head()

,Chapter,Section,Name,Text,IsRepealed,Details,Part
0,90,1,Definitions,Section 1. The following words used in this ch...,False,http://malegislature.gov/api/Chapters/90/Secti...,"{'Code': 'I', 'Details': 'http://malegislature..."
1,90,1A,Registration of motor vehicles; liability insu...,"Section 1A. No motor vehicle or trailer, excep...",False,http://malegislature.gov/api/Chapters/90/Secti...,"{'Code': 'I', 'Details': 'http://malegislature..."
2,90,1B,Motorized bicycles; operation regulations,Section 1B. A motorized bicycle shall not be o...,False,http://malegislature.gov/api/Chapters/90/Secti...,"{'Code': 'I', 'Details': 'http://malegislature..."
3,90,1C,Motorized bicycles and scooters; compliance wi...,Section 1C. Motorized bicycles and motorized s...,False,http://malegislature.gov/api/Chapters/90/Secti...,"{'Code': 'I', 'Details': 'http://malegislature..."
4,90,1D,Motorized bicycles; sales; number sticker or p...,Section 1D. Any person who is engaged in the b...,False,http://malegislature.gov/api/Chapters/90/Secti...,"{'Code': 'I', 'Details': 'http://malegislature..."


In [73]:
full_df['Part'] = full_df['Part'].apply(str)

KeyError: 'Part'

In [74]:
full_df.columns

Index(['Chapter', 'Section', 'Name', 'Text', 'IsRepealed', 'Details'], dtype='object')

In [61]:
full_df.to_csv('pulled_mgl.csv', index=False)

In [69]:
full_df.to_excel('pulled_mgl.xlsx', index=False)


In [86]:
from sqlalchemy import create_engine

# Replace these variables with your database credentials
username = 'root'
password = 'password123'
host = 'localhost'
port = '3306'
database = 'watchCommander'

# Create an engine to connect to MySQL
# The format is: "mysql+pymysql://<username>:<password>@<host>:<port>/<database>"
engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}:{port}/{database}")


In [87]:
# Example DataFrame

# Replace 'your_table_name' with the desired table name
table_name = 'mgl'

# Export the DataFrame to MySQL
# If the table doesn't exist, it will be created automatically
# full_df.to_sql(table_name, con=engine, index=False, if_exists='replace')

full_df.to_sql(name=table_name, con=engine, if_exists='append', index=False)



1142

In [84]:
512000000 * 2

1024000000

In [14]:
# pull raw string of pd's to get formatted into a list, dropping extra characters
raw_string = """
    Abington Police Department
    Acton Police Department
    Acushnet Police Department
    Adams Police Department[7]
    Agawam Police Department[8]
    Alford Police Department
    Andover Police Department
    Amesbury Police Department
    Amherst Police Department
    Arlington Police Department
    Ashburnham Police Department
    Ashby Police Department
    Ashfield Police Department
    Ashland Police Department
    Aquinnah Police Department
    Athol Police Department
    Attleboro Police Department[9]
    Auburn Police Department
    Avon Police Department
    Ayer Police Department
    Barnstable Police Department[10]
    Barre Police Department
    Becket Police Department
    Bedford Police Department[11]
    Belchertown Police Department
    Bellingham Police Department
    Belmont Police Department
    Berkley Police Department[citation needed]
    Berlin Police Department[citation needed]
    Bernardston Police Department
    Beverly Police Department
    Billerica Police Department
    Blackstone Police Department[citation needed]
    Blandford Police Department[12]
    Bolton Police Department
    Boston Police Department[13]
    Bourne Police Department[14]
    Boxborough Police Department
    Boxford Police Department
    Boylston Police Department
    Braintree Police Department[15]
    Brewster Police Department
    Bridgewater Police Department
    Brimfield Police Department
    Brockton Police Department[16]
    Brookfield Police Department
    Brookline Police Department
    Buckland Police Department
    Burlington Police Department
    Cambridge Police Department[17]
    Canton Police Department
    Carlisle Police Department
    Carver Police Department
    Charlemont Police Department
    Charlton Police Department
    Chatham Police Department
    Chelmsford Police Department
    Chelsea Police Department[18]
    Cheshire Police Department
    Chester Police Department
    Chesterfield Police Department
    Chicopee Police Department[19]
    Chilmark Police Department
    Clarksburg Police Department
    Clinton Police Department
    Cohasset Police Department
    Colrain Police Department
    Concord Police Department
    Conway Police Department
    Cummington Police Department
    Dalton Police Department
    Danvers Police Department
    Dartmouth Police Department
    Dedham Police Department
    Deerfield Police Department
    Dennis Police Department
    Dighton Police Department
    Douglas Police Department
    Dover Police Department
    Dracut Police Department
    Dudley Police Department
    Dunstable Police Department
    Duxbury Police Department
    East Bridgewater Police Department
    East Brookfield Police Department
    East Longmeadow Police Department
    Eastham Police Department
    Easthampton Police Department
    Easton Police Department
    Edgartown Police Department
    Egremont Police Department
    Erving Police Department
    Essex Police Department
    Everett Police Department[20]
    Fairhaven Police Department
    Fall River Police Department[21]
    Falmouth Police Department[22]
    Fitchburg Police Department[23]
    Foxborough Police Department
    Framingham Police Department
    Franklin Police Department[24]
    Freetown Police Department
    Gardner Police Department
    Georgetown Police Department
    Gill Police Department
    Gloucester Police Department
    Goshen Police Department
    Gosnold Police Department
    Grafton Police Department
    Granby Police Department
    Granville Police Department
    Great Barrington Police Department
    Greenfield Police Department
    Groton Police Department
    Hadley Police Department
    Halifax Police Department
    Hamilton Police Department
    Hampden Police Department
    Hancock Police Department
    Hanover Police Department
    Hanson Police Department
    Hardwick Police Department
    Harvard Police Department
    Harwich Police Department
    Hatfield Police Department
    Haverhill Police Department[25]
    Hawley Police Department
    Heath Police Department
    Hingham Police Department
    Hinsdale Police Department
    Holbrook Police Department
    Holden Police Department
    Holland Police Department
    Holliston Police Department
    Holyoke Police Department[26]
    Hopedale Police Department
    Hopkinton Police Department
    Hubbardston Police Department
    Hudson Police Department
    Hull Police Department[27]
    Huntington Police Department
    Ipswich Police Department
    Kingston Police Department
    Lancaster Police Department
    Lanesborough Police Department
    Lakeville Police Department
    Lawrence Police Department[28]
    Lee Police Department[29]
    Leicester Police Department
    Lenox Police Department
    Leominster Police Department
    Leverett Police Department
    Lexington Police Department
    Leyden Police Department
    Lincoln Police Department
    Littleton Police Department
    Longmeadow Police Department
    Lowell Police Department[30]
    Ludlow Police Department
    Lunnenburg Police Department
    Lynn Police Department[31]
    Lynnfield Police Department
    Malden Police Department[32]
    Manchester-by-the-Sea Police Department
    Mansfield Police Department
    Marion Police Department
    Marblehead Police Department
    Marlborough Police Department[33]
    Marshfield Police Department
    Mashpee Police Department[citation needed]
    Mattapoisett Police Department[citation needed]
    Maynard Police Department
    Medfield Police Department
    Medford Police Department[34]
    Medway Police Department
    Melrose Police Department[35]
    Mendon Police Department
    Merrimac Police Department[citation needed]
    Methuen Police Department[36]
    Middleborough Police Department
    Middlefield Police Department
    Middleton Police Department
    Milford Police Department
    Millbury Police Department[37]
    Millis Police Department
    Milleville Police Department
    Milton Police Department
    Monson Police Department
    Montague Police Department
    Monterey Police Department
    Nahant Police Department
    Nantucket Police Department
    Natick Police Department
    Needham Police Department
    New Bedford Police Department[38]
    New Braintree Police Department
    New Marlborough Police Department
    New Salem Police Department
    Newbury Police Department
    Newburyport Police Department
    Newton Police Department[39]
    Norfolk Police Department
    North Adams Police Department[40]
    North Andover Police Department
    North Attleborough Police Department
    North Brookfield Police Department
    North Reading Police Department
    Northampton Police Department[41]
    Northborough Police Department
    Northbridge Police Department
    Northfield Police Department
    Norton Police Department
    Norwell Police Department[42]
    Norwood Police Department
    Oak Bluffs Police Department
    Oakham Police Department
    Orange Police Department
    Orleans Police Department
    Otis Police Department
    Oxford Police Department
    Palmer Police Department
    Paxton Police Department
    Peabody Police Department[43]
    Pepperell Police Department
    Pelham Police Department
    Pembroke Police Department
    Peru Police Department
    Petersham Police Department
    Phillipston Police Department
    Pittsfield Police Department[44]
    Plainfield Police Department
    Plainville Police Department
    Plymouth Police Department
    Plympton Police Department
    Princeton Police Department
    Provincetown Police Department[45]
    Quincy Police Department[46]
    Raynham Police Department
    Randolph Police Department
    Reading Police Department
    Rehoboth Police Department
    Revere Police Department[47]
    Richmond Police Department
    Rochester Police Department
    Rockland Police Department
    Rockport Police Department
    Rowe Police Department
    Rowley Police Department
    Royalston Police Department
    Russell Police Department
    Rutland Police Department
    Salem Police Department[48]
    Salisbury Police Department
    Sandisfield Police Department
    Sandwich Police Department[49]
    Saugus Police Department
    Savoy Police Department
    Scituate Police Department
    Seekonk Police Department[50]
    Sharon Police Department
    Sheffield Police Department
    Shelburne Police Department
    Sherborne Police Department
    Shirley Police Department
    Shrewsbury Police Department
    Shutesbury Police Department
    Somerset Police Department
    Somerville Police Department[51]
    South Hadley Police Department
    Southborough Police Department
    Southbridge Police Department
    Southampton Police Department
    Southwick Police Department
    Spencer Police Department
    Springfield Police Department[52]
    Sterling Police Department
    Stockbridge Police Department
    Stoneham Police Department
    Stoughton Police Department
    Stow Police Department
    Sturbridge Police Department
    Sudbury Police Department
    Sutton Police Department
    Swansea Police Department
    Swampscott Police Department
    Taunton Police Department[53]
    Templeton Police Department
    Tewksbury Police Department[54]
    Tisbury Police Department
    Tolland Police Department
    Topsfield Police Department
    Townsend Police Department
    Truro Police Department[55]
    Tynsborough Police Department
    Tyringham Police Department
    Upton Police Department
    Uxbridge Police Department
    Wakefield Police Department
    Wales Police Department
    Walpole Police Department
    Waltham Police Department[56]
    Ware Police Department
    Wareham Police Department[57]
    Warren Police Department
    Warwick Police Department
    Washington Police Department
    Watertown Police Department[58]
    Wayland Police Department
    Webster Police Department
    Wellesley Police Department
    Wellfleet Police Department[59]
    Wendell Police Department
    Wenham Police Department
    West Boylston Police Department
    West Bridgewater Police Department
    West Brookfield Police Department
    West Newbury Police Department
    West Springfield Police Department[60]
    West Stockbridge Police Department
    West Tisbury Police Department
    Westborough Police Department
    Westhampton Police Department
    Westfield Police Department[61]
    Westford Police Department
    Westminster Police Department
    Westport Police Department
    Weston Police Department
    Westwood Police Department
    Weymouth Police Department[62]
    Whately Police Department
    Whitman Police Department
    Wilbraham Police Department
    Williamsburg Police Department
    Williamstown Police Department
    Wilmington Police Department
    Winchendon Police Department
    Winchester Police Department
    Windsor Police Department
    Winthrop Police Department
    Woburn Police Department[63]
    Worcester Police Department[64]
    Worthington Police Department
    Wrentham Police Department
    Yarmouth Police Department[65]"""

In [15]:
lines = [line.strip() for line in raw_string.split('\n') if line.strip()]
town_police_departments = [line for line in lines if line.endswith(" Police Department")]